Necessary imports

In [1]:
%pip install pandas
%pip install plotly
%pip install scipy

from scipy import stats
import pandas as pd
import plotly.express as px

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 6.4 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 2.6 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


Carregamento de dados

A partir do site que pegamos o arquivo (http://dados.recife.pe.gov.br/pt_BR/dataset/servico-de-atendimento-movel-de-urgencia-samu-2022/resource/7c005351-2d14-46cf-b2f0-6622a5553b30), os tipos dos dados são:

data: timestamp
hora_minuto: text municipio: text
bairro: text
endereco: text
origem_chamado: text
tipo: text
subtipo: text
sexo: text
idade: numeric
motivo_finalizacao: text
motivo_desfecho: text

In [3]:
dtypes = {
    "hora_minuto":  str,
    "municipio": "category",
    "bairro": "category",
    "endereco": "category",
    "origem_chamado": "category",
    "tipo": "category",
    "subtipo": "category",
    "sexo": "category",
    "idade": float,
    "motivo_finalizacao": "category",
    "motivo_desfecho": "category",
}

columns_to_datetime = ["data"]

raw_df = pd.read_csv("./datasets/ocorrencias2022.csv", sep=';', dtype=dtypes, parse_dates=columns_to_datetime)
raw_df

,data,hora_minuto,municipio,bairro,endereco,origem_chamado,tipo,subtipo,sexo,idade,motivo_finalizacao,motivo_desfecho
0,2022-01-01,00:02:19,RECIFE,JARDIM SAO PAULO,R LEANDRO BARRETO,RESIDENCIAL,RESPIRATORIA,CASO SUSPEITO COVID-19,FEMININO,81.0,NaN,PACIENTE RECUSA SER REMOVIDO
1,2022-01-01,00:03:00,RECIFE,MADALENA,R ALTINHO,RESIDENCIAL,CAUSAS EXTERNAS,QUEDA DA PROPRIA ALTURA,FEMININO,81.0,NaN,DESISTÊNCIA DA SOLICITAÇÃO
2,2022-01-01,00:09:38,MACAPARANA,CENTRO,R POSSIDONIO JULIAO DA SILVA,VIA PÚBLICA,CAUSAS EXTERNAS,OUTROS,MASCULINO,0.0,REGULAÇÃO POR TELEFONE,SEM DESFECHO
3,2022-01-01,00:10:02,VITORIA DE SANTO ANTAO,CENTRO,RUA JOSE COSTA N 93 BAIRRO DA,RESIDENCIAL,GERAIS/OUTROS,OUTROS,FEMININO,NaN,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
4,2022-01-01,00:12:16,PALMARES,PALMARES CENTRO,RUA DOUTOR COSTA MAIA,RESIDENCIAL,GERAIS/OUTROS,OUTROS,MASCULINO,84.0,NaN,OCORRÊNCIA CONCLUÍDA COM ÊXITO
...,...,...,...,...,...,...,...,...,...,...,...,...
94685,2022-06-30,23:50:01,RECIFE,IPUTINGA,R PEDRO MAX,RESIDENCIAL,CAUSAS EXTERNAS,OUTROS,FEMININO,48.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO
94686,2022-06-30,23:51:07,JABOATAO DOS GUARARAPES,GUARARAPES,R ANDURINHA,RESIDENCIAL,RESPIRATORIA,OUTROS,FEMININO,74.0,NaN,OCORRÊNCIA CONCLUÍDA COM ÊXITO
94687,2022-06-30,23:52:53,PAULISTA,PARATIBE,BR 101 SENTIDO ABREU E LIMA,VIA PÚBLICA,CAUSAS EXTERNAS,ATROPELAMENTO POR CARRO,MASCULINO,0.0,NaN,OCORRÊNCIA CONCLUÍDA COM ÊXITO
94688,2022-06-30,23:53:34,RECIFE,IPUTINGA,RUA ALMIRANTE PEDRO MARQUES,RESIDENCIAL,GERAIS/OUTROS,OUTROS,FEMININO,48.0,DESISTÊNCIA DA SOLICITAÇÃO,SEM DESFECHO


Considerações iniciais sobre o dataset

O dataset descreve ligações para o Serviço de Atendimento Móvel de Urgência (SAMU), realizados no ano de 2022.
Possui informações que:
Identificam o local e hora onde uma ocorrência foi registrada;
sexo e idade que identificam o indivíduo responsável pelo chamdado, 

In [4]:
raw_df.describe()

,idade
count,93764.000000
mean,47.013886
std,25.460475
min,0.000000
25%,28.000000
50%,47.000000
75%,67.000000
max,747.000000


In [ ]:
raw_df.dtypes

In [5]:
raw_df.isna().sum()

data                      0
hora_minuto               0
municipio                 3
bairro                    4
endereco               1914
origem_chamado            0
tipo                      0
subtipo                  17
sexo                   1399
idade                   926
motivo_finalizacao    60456
motivo_desfecho           0
dtype: int64

Mais da metade do dataset está com a coluna "motivo_finalizacao" vazia, será impraticável utilizar essa coluna na nossa análise, portanto removeremos ela.

para o restante das colunas, nós removeremos as instâncias que contém alguma coluna vazia, com exceção da coluna de endereço, já que ela não vai ser tão útil para a análise que queremos.

A remoção dessas instâncias não vão fazer muita diferença já que a quantidade das linhas removidas são ínfimas, (menos de 5%), em comparação à quantidade total de dados do dataset.

Tratando dados vazios

Grande parte dos motivos de finalização estão vazios.
Sexo e idade tem um número relevante de valores faltantes, então utilizamos a função fillna(), com o método forward fill.

Com base nessa distribuição de valores faltantes, decidimos focar as análises em 

In [ ]:
raw_df = raw_df.drop("motivo_finalizacao", axis=1)
raw_df = raw_df.dropna(subset=["municipio", "bairro", "subtipo", "sexo", "idade"])
raw_df.isna().sum()

In [ ]:
raw_df["idade"].describe()

ter a idade máxima de 747 é impossível, uma passada no google diz que há pessoas com certificados de nascimento entre 115 anos até 121 anos, portanto nós escolhemos definir uma idade máxima de 120 anos para deixar mais fácil de discretizar a idade em uma série de 6 categorias.

In [ ]:
def older_than_120(age: int):
  if age >= 120:
    return 120
  return age

raw_df["idade"] = raw_df["idade"].apply(lambda x : older_than_120(x))
raw_df["idade"].describe()

In [ ]:
raw_df["faixa_etaria"] = pd.cut(raw_df["idade"], 6)
raw_df["faixa_etaria"]

Teste de hipótese:

In [ ]:
idade_ascending = raw_df.groupby("municipio")["idade"].mean().sort_values(ascending=True)
idade_ascending.plot.box()

In [ ]:
idade_ascending.plot.hist()

In [ ]:
stats.shapiro(idade_ascending)[1]